# Use GLM-4 API to fine-tune model with LORA method

**This tutorial is available in English and is attached below the Chinese explanation**

此代码将展示如何调用 GLM-4 API 来使用LORA的方法对一个 glm 模型进行微调。你将会在本代码中学到：
1. 微调数据集的格式
2. 如何上传微调的数据集
3. 如何管理和查看自己的微调数据集
4. 如何设置超参数并开始微调模型
5. 如何调用微调后的模型

This code will show how to call the GLM-4 API to fine-tune a glm model using LORA methods. What you will learn in this code:
1. Fine-tune the format of the data set
2. How to upload fine-tuned dataset
3. How to manage and view your own fine-tuning data set
4. How to set hyperparameters and start fine-tuning the model
5. How to call the fine-tuned model

## 1. Set API Keys and Client
首先，我们设置好 API KEY，并将这个API KEY设置为环境变量，这样我们就可以在代码中直接调用了。

First, we set up the API KEY and set this API KEY as an environment variable so that we can call it directly in the code.

In [1]:
import os
from zhipuai import ZhipuAI

os.environ["ZHIPUAI_API_KEY"] = ""
client = ZhipuAI()

## 2. Prepare the dataset

需要准备训练的数据集，并将其拆分为训练集和验证集。根据官方文档的文件规则，我们需要将要整理完后的数据集格式如下：

We need to prepare a training dataset and split it into a training set and a validation set. According to the file rules of the official document, we need to format the dataset to be organized as follows:
```json lines
{"messages": [{"role": "user", "content": "你是专门进行实体抽取的专家。请从input中抽取出符合schema定义的实体，不存在的实体类型返回空列表。请按照JSON字符串的格式回答。schema:['address','book','company','game','government','movie']，input:中新网12日22日电据中国银监会网站消息，日前，银监会通知印发了《银行与信托公司业务合作指引》，"}, {"role": "assistant", "content": "{\"address\":[],\"book\":[\"中新网\",\"《银行与信托公司业务合作指引》\"],\"company\":[],\"game\":[],\"government\":[\"中国银监会\",\"银监会\"],\"movie\":[]}"}]}
{"messages": [{"role": "user", "content": "你是专门进行实体抽取的专家。请从input中抽取出符合schema定义的实体，不存在的实体类型返回空列表。请按照JSON字符串的格式回答。schema:['name','organization','position','scene']，input:此前，住房和城乡建设部副部长齐骥在谈及保障性住房需要的几大制度及政府保障工作时，"}, {"role": "assistant", "content": "{\"name\":[\"齐骥\"],\"organization\":[],\"position\":[\"副部长\"],\"scene\":[]}"}]}
```

在这里，我们使用[实体识别数据集](./data/Reco)作为一个数据集的例子。数据集已经被分割为 `train.jsonl` 和 `dev.jsonl`。可直接用于训练。
Here, we use the [Entity Recognition Dataset](./data/Reco) as an example of a dataset. The dataset has been split into `train. jsonl` and `dev. jsonl`. Can be directly used for training.


我们可以通过API开放平台，或通过SDK提供的接口上传数据集到Zhipu AI的服务器上，并查看上传的数据集的文件列表。

After the data set is converted, upload the data set to Zhipu AI's server through the interface provided by the SDK, and view the file list of the uploaded data set.

In [2]:
client.files.create(
    file=open("data/Reco/train.jsonl", "rb"),
    purpose="fine-tune"
)

client.files.create(
    file=open("data/Reco/dev.jsonl", "rb"),
    purpose="fine-tune"
)

ListOfFileObject(object='list', data=[FileObject(id='file-20240821061501259-6ksl5', bytes=105064, created_at=1724220901000, filename='dev.jsonl', object='fd', purpose='fine-tune', status=None, status_details=None), FileObject(id='file-20240821061500706-kbn67', bytes=2623788, created_at=1724220901000, filename='train.jsonl', object='fd', purpose='fine-tune', status=None, status_details=None), FileObject(id='file-20240821061335629-xlcf7', bytes=2623788, created_at=1724220816000, filename='train.jsonl', object='fd', purpose='fine-tune', status=None, status_details=None), FileObject(id='file-20240812081734566-x7fgx', bytes=1278, created_at=1723450655000, filename='demo.jsonl', object='fd', purpose='fine-tune', status=None, status_details=None), FileObject(id='file-20240812081734093-lc42d', bytes=1278, created_at=1723450654000, filename='demo.jsonl', object='fd', purpose='fine-tune', status=None, status_details=None), FileObject(id='file-20240812081726911-f9xc8', bytes=1278, created_at=1723

In [4]:
client.files.list()

ListOfFileObject(object='list', data=[FileObject(id='file-20240821061500706-kbn67', bytes=2623788, created_at=1724220901000, filename='train.jsonl', object='fd', purpose='fine-tune', status=None, status_details=None), FileObject(id='file-20240821061501259-6ksl5', bytes=105064, created_at=1724220901000, filename='dev.jsonl', object='fd', purpose='fine-tune', status=None, status_details=None), FileObject(id='file-20240724054219665-z6ml9', bytes=58025, created_at=1721799740000, filename='math_dev.jsonl', object='fd', purpose='fine-tune', status=None, status_details=None), FileObject(id='file-20240724054219205-qv45x', bytes=337440, created_at=1721799739000, filename='math_train.jsonl', object='fd', purpose='fine-tune', status=None, status_details=None), FileObject(id='file-20240410071801863-92dd2', bytes=777692, created_at=1712733482000, filename='output_3.jsonl', object='fd', purpose='fine-tune', status=None, status_details=None), FileObject(id='file-20240410055723302-8gmjz', bytes=9347, 

## 3. Start training using the uploaded file

现在，在列表中已经找到了训练集和验证集的两个文件，我们设置好相关的参数，并上传到智谱AI的服务器上开始训练。按照[官方文档](https://open.bigmodel.cn/dev/api#finetuning)填写相关参数。我们就可以开训练了。我选择使用模型 `glm-4-0520` 进行训练。该方法跟直接在智谱AI平台上可视化训练效果相同。

当前版本 SDK 中，默认使用的是 LORA 的方式进行训练。由于超参数在文档中仅开放了三个参数，分别是:
- `learning_rate_multiplier` 学习率的倍数
- `batch_size` 批次大小
- `n_epochs` 训练轮数

接着，我们就能开始训练了，并且，我们打印了训练的`job_id`，以便后续查看训练进度。


Now that we have found the two files of the training set and validation set in the list, we set the relevant parameters and uploaded them to the Zhipu AI server to start training. Fill in the relevant parameters according to [official documentation](https://open.bigmodel.cn/dev/api#finetuning). We can start training. I chose to use the smallest model `chatglm3-6b` for training.

In the current version of the SDK, the LORA method is used for training by default. Since the hyperparameters only open three parameters in the document, they are
- `learning_rate_multiplier` multiple of the learning rate
- `batch_size` batch size
- `n_epochs` Number of training epochs
These values have default values in the documentation, but I chose to set them myself.


In [11]:
job = client.fine_tuning.jobs.create(
    model="glm-4-air",
    training_file="file-20240821061500706-kbn67",
    validation_file="file-20240821061501259-6ksl5",
    hyperparameters={
        "learning_rate_multiplier": 0.5,
        "batch_size": 32,
        "n_epochs": 2,
    },
    suffix="zr_reco",
)
job_id =job.id
job_id

'ftjob-20240821154132388-khlc6'

## 4. Check the training progress

我们可以查看这个训练任务的进度，以及训练的状态。 由于训练任务是异步的，因此，我们需要等待一段时间后才能看到训练的状态。其中`status` 训练状态，现在是 `running`，表示正在训练

由于本数据集的训练机数量较大（超过11万条数据），训练时间较长（在书写本cookbook时微调时间超过了1个小时), 因此，我在实际微调的过程使用，使用的是 `dev.jsonl` 文件进行微调和验证
您可以自己删除部分数据集，这样能有效降低训练时间和训练的token消耗成本。

We can check the progress of this training task and the status of training. Since the training task is asynchronous, we need to wait for a period of time before we can see the status of the training. in
`status` training status, now `running`, indicating training is in progress

Since the number of training machines in this data set is large (more than 110,000 pieces of data) and the training time is long (the fine-tuning time took more than 1 hour when writing this cookbook), therefore, in the actual fine-tuning process, I used `dev.jsonl` file for fine-tuning and verification
You can delete part of the data set yourself, which can effectively reduce training time and training token consumption costs.

In [12]:
fine_tuning_job = client.fine_tuning.jobs.retrieve(fine_tuning_job_id=job_id)
fine_tuning_job

FineTuningJob(id='ftjob-20240821154132388-khlc6', request_id=None, created_at=1724226092, error=None, fine_tuned_model='glm-4-air:1601347751:zr_reco:a8ostspv', finished_at=None, hyperparameters=None, model='glm-4-air', object='fine_tuning.job', result_files=[], status='created', trained_tokens=None, training_file='file-20240821061500706-kbn67', validation_file='file-20240821061501259-6ksl5', deployButton=False, experienceCenterButton=False)

通过等待一段时间我们再次查询，我们发现，模型已经训练完成，接下来，我们就是调用模型进行测试了。
通过 Python SDK来查询我所有的微调任务进度，其中，这些字段的含义如下：
- `id` 微调任务的id
- `fine_tuned_model` 微调模型的名称
- `request_id` 用户请求的id，默认是空
- `created_at`,`finished_at` 任务的开始时间和结束时间,在这里都是空
- `status` 任务的状态，现在是 `running`，表示正在训练

By waiting for a while and querying again, we find that the model has been trained. Next, we call the model for testing.
Use the Python SDK to check the progress of all my fine-tuning tasks.

In [9]:
client.fine_tuning.jobs.list()

ListOfFineTuningJob(object='list', data=[FineTuningJob(id='ftjob-20240821141947481-r7ln2', request_id=None, created_at=1724221187, error=None, fine_tuned_model='glm-4-flash:1601347751:zr_reco:7nnzj3pw', finished_at=1724221782, hyperparameters=None, model='glm-4-flash', object='fine_tuning.job', result_files=[], status='succeeded', trained_tokens=938732, training_file='file-20240821061500706-kbn67', validation_file='file-20240821061501259-6ksl5', running_at=1724221230), FineTuningJob(id='ftjob-20240724150633046-qxsr4', request_id=None, created_at=1721804793, error=None, fine_tuned_model='glm-4-0520:1601347751::k8apzh6j', finished_at=1721806698, hyperparameters=None, model='glm-4-0520', object='fine_tuning.job', result_files=[], status='succeeded', trained_tokens=233736, training_file='file-20240724054219205-qv45x', validation_file='file-20240724054219665-z6ml9', running_at=1721804841)], has_more=None)

## 5. Using the model after fine-tuning

在完成微调任务之后，我们会得到一个新的模型，我们可以通过SDK来完成对这个模型的调用。我们首先查看一下模型列表，然后选择最新的模型进行调用。使用微调后的模型的请求方式与常规的 GLM 模型没有区别，仅需将 `model` 更换为刚才微调的模型名称即可。

After completing the fine-tuning task, we will get a new model, and we can complete the call to this model through the SDK. We first check the model list, and then select the latest model to call. The request method for using the fine-tuned model is no different from the regular GLM model. You only need to replace `model` with the name of the model you just fine-tuned.


In [10]:
response = client.chat.completions.create(
    model="glm-4-flash:1601347751:zr_reco:7nnzj3pw",
    messages=[{"role": "user",
               "content": "你是专门进行实体抽取的专家。请从input中抽取出符合schema定义的实体，不存在的实体类型返回空列表。请按照JSON字符串的格式回答。schema:['name','organization','position','scene']，input:结果升班马热那亚首回合做客0比0逼和了桑普，但在主场却遭遇最后3分钟，最终以0比1惜败。"}]

)
response.choices[0].message.content

'{"name": ["热那亚", "桑普"], "organization": [], "position": [], "scene": ["首回合", "主场", "最后3分钟"]}'